# Creating simulated data from a mosaic image

This notebook demonstrates how to use Mirage to create simulated data from a distortion-free mosaic image. In this case, we will use a mosaic of the GOODS-S region from the [CANDELS survey](https://archive.stsci.edu/prepds/candels/). Several stamp images containing additional objects will also be used. These images are cut-outs from a recent [HST/WFC3 SNAPSHOT program of some 3C sources](https://hz3c.stsci.edu/Observations.html). 

For each observation to be simulated, the appropriate area of the mosaic is extracted from the full mosaic, and is resampled in order to introduce the distortion associated with the JWST instrument to be used. This distorted image is then addded to the simulated data in one of two ways.

If you wish to modify the image from the mosaic in any way, such as adding additional objects or scaling the brightness, then the mosaic image can be added to one of Mirage's "extended" source catalogs, along with additional sources.

If you do not wish to modify the cropped mosaic image in any way (other than introducing the appropriate distortion), then the distorted image can be used directly as a seed image, and you only need to run the dark_prep and obs_generation steps of Mirage in order to create the final simulated data.

## Table of contents

* [Imports](#imports)
* [Download Data](#download)
* [Using resampled image in an extended source catalog](#resample_into_catalog)
    * [Provide the PSF FWHM in the mosaic data](#provide_fwhm)
    * [Measure the FWHM by fitting a 2D Gaussian](#measure_fwhm)
    * [Run yaml_generator to create Mirage input yaml files](#yaml_generator_catalogs)
    * [Source Catalogs](#source_catalogs)
    * [Extract images from mosaic, resample, and add to catalog](#crop_and_blot_catalog)
    * [Create the simulated data](#create_data_catalog)
    * [Look at simulated data](#examine_data_catalog)
* [Use resampled image as a seed image](#resample_seed)
    * [Run yaml_generator to create Mirage input yaml files](#yaml_generator_seed)
    * [Create the simulted data](#create_data_seed)
    * [Look at simulated data](#examine_data_seed)

---
<a id='imports'></a>
## Imports

In [ ]:
import os
import yaml

In [ ]:
from astropy.io import fits
from astropy.modeling import models, fitting
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from mirage.catalogs.catalog_generator import ExtendedCatalog
from mirage.catalogs.create_catalog import combine_catalogs
from mirage.dark.dark_prep import DarkPrep
from mirage.ramp_generator.obs_generator import Observation
from mirage.imaging_simulator import ImgSim
from mirage.reference_files.downloader import download_file
from mirage.seed_image.fits_seed_image import ImgSeed
from mirage.yaml import yaml_generator

---
<a id='download'></a>
## Download Data

Download FITS files containing the mosaic, as well as three small stamp images containing single objects. 

In [ ]:
base_url = ('https://data.science.stsci.edu/redirect/JWST/jwst-simulations/'
            'mirage_reference_files/example_data_for_notebooks')

In [ ]:
stamp_files = ['3C305.1_stamp.fits', '3C324_stamp.fits', '3C454.1_stamp.fits',
               'hlsp_candels_hst_acs_gs-tot-sect23_f814w_v1.0_drz.fits']

In [ ]:
for stamp_file in stamp_files:
    stamp_url = '{}/{}'.format(base_url, stamp_file)
    s_file = download_file(stamp_url, stamp_file, output_directory='./')

---
<a id='resample_into_catalog'></a>
## Using resampled image in an extended source catalog

In [ ]:
mosaicfile = 'hlsp_candels_hst_acs_gs-tot-sect23_f814w_v1.0_drz.fits'
xml_file = 'extended_object_test.xml'
pointing_file = xml_file.replace('.xml', '.pointing')

---
<a id='provide_fwhm'></a>
### Provide the PSF FWHM in the mosaic data

In [ ]:
# From the CANDELS documentation
mosaic_fwhm = 0.09  # arcseconds

---
<a id='measure_fwhm'></a>
### Alternatively, measure the FWHM by fitting a 2D Gaussian

In [ ]:
mosaic = fits.getdata(mosaicfile)
mosaic_header = fits.getheader(mosaicfile)

In [ ]:
# Extract a subimage around a star
box = mosaic[3254: 3330, 7071: 7153]
yp, xp = box.shape

In [ ]:
# Look at the extracted box. Make sure the PSF looks reasonable
plt.imshow(box)

In [ ]:
# Generate grid of same size as box, to be used in fitting
y, x, = np.mgrid[:yp, :xp]

In [ ]:
# Fit the model
p_init = models.Gaussian2D()
fit_p = fitting.LevMarLSQFitter()
fitted_psf = fit_p(p_init, x, y, box)

In [ ]:
# Fit results. The FWHM is needed as an input to the
print('Amplitude: ', fitted_psf.amplitude.value)
print('X_mean: ', fitted_psf.x_mean.value)
print('Y_mean: ', fitted_psf.y_mean.value)
print('X_FWHM: ', fitted_psf.x_fwhm)
print('Y_FWHM: ', fitted_psf.y_fwhm)
print('X_stddev: ', fitted_psf.x_stddev.value)
print('Y_stddev: ', fitted_psf.y_stddev.value)

In [ ]:
measured_mosaic_fwhm = fitted_psf.y_fwhm * (np.abs(mosaic_header['CD2_2']) * 3600.)

In [ ]:
# Measured FWHM in arcseconds
measured_mosaic_fwhm

In [ ]:
mosaic_fwhm = measured_mosaic_fwhm

---
<a id='yaml_generator_catalogs'></a>
### Run yaml_generator to create Mirage input yaml files

User-inputs to the yaml generator. Note that you can still use a catalogs input here and add
point sources or galaxies. Extended source catalog names will be added later

In [ ]:
cr = {'library': 'SUNMAX', 'scale': 1.0}
dates = '2019-5-25'
background = 'low'
pav3 = 0.0
#catalogs = {'NGC1234': {'nircam': {'point_source': 'ngc1234_ptsrc_nrc.cat',
#                                   'galaxy': 'ngc1234_galaxy_nrc.cat',
#                                   }
#                        }
#            }

Run the yaml generator

In [ ]:
yam = yaml_generator.SimInput(xml_file, pointing_file, verbose=True,
                              output_dir='yamls',
                              cosmic_rays=cr,
                              #catalogs=catalogs,
                              background=background, roll_angle=pav3, dates=dates,
                              simdata_output_dir='simdata',
                              datatype='raw')
yam.use_linearized_darks = True
yam.create_inputs()

---
<a id='source_catalogs'></a>
### Source catalogs

Get a list of all instruments, apertures, and filters used in the APT file

In [ ]:
instruments = yam.info['Instrument']
filter_keywords = ['FilterWheel', 'ShortFilter', 'LongFilter', 'Filter']
pupil_keywords = ['PupilWheel', 'ShortPupil', 'LongPupil']
yam.info

nrc_sw_optics = set([(f, p) for f, p in zip(yam.info['ShortFilter'], yam.info['ShortPupil'])])
nrc_lw_optics = set([(f, p) for f, p in zip(yam.info['LongFilter'], yam.info['LongPupil'])])
niriss_optics = set([(f, p) for f, p in zip(yam.info['FilterWheel'], yam.info['PupilWheel'])])
niriss_wfss_optics = set([(f, p) for f, p in zip(yam.info['Filter'], yam.info['PupilWheel'])])

print('NIRCam filters/pupils used in this proposal: ')
print(nrc_sw_optics)
print(nrc_lw_optics)
print('\nNIRISS filters/pupils used in this proposal: ')
print(niriss_optics)
print(niriss_wfss_optics)
print(('\nBe sure to add magnitude columns to the template catalog '
        'for all filters you are going to simulate.\n'))

#### Create extended source catalog

Create a template extended source catalog containing sources other than the mosaic image that you want to add to the seed image. The resampled mosaic will be added to this template later. Note that you must add magnitude values for these other sources in all filters that are used in the proposal.

If you do not have any extended sources other than the mosaic, set the template_cat to None, so that later we know there is nothing to combine with the catalog containing the mosaic data.

In [ ]:
template_cat = None

If you do have extended sources in addition to the mosaic image, create template_cat here and add those sources.

In [ ]:
filter1 = 'F150W'
filter2 = 'F444W'

In [ ]:
other_stamp_files = ['3C305.1_stamp.fits', '3C324_stamp.fits', '3C454.1_stamp.fits']
other_stamp_ra = [53.164375, 53.168375, 53.160375]
other_stamp_dec = [-27.815355, -27.811355, -27.819355]
other_stamp_pa = [0., 0., 0.]
other_stamp_f150w_mags = [18., 19., 19.5]
other_stamp_f444w_mags = [22.5, 23.5, 24.0]

# Magnitude values must be strings here because we will be combining them
# with values of 'None' for the resampled image magnitudes
f150w_mags_as_str = [str(element) for element in other_stamp_f150w_mags]
f444w_mags_as_str = [str(element) for element in other_stamp_f444w_mags]

template_extended_catalog_file = 'extended_sources_template.cat'
template_cat = ExtendedCatalog(filenames=other_stamp_files, ra=other_stamp_ra, dec=other_stamp_dec,
                               position_angle=other_stamp_pa)
template_cat.add_magnitude_column(f150w_mags_as_str, instrument='nircam', filter_name=filter1)
template_cat.add_magnitude_column(f444w_mags_as_str, instrument='nircam', filter_name=filter2)
template_cat.save(template_extended_catalog_file)

---
<a id='crop_and_blot_catalog'></a>
### Extract images from mosaic, resample, and add to catalog

In this step, crop a roughly detector-sized subarray from the mosaic image at the location specified in the yaml file. Convolve the subarray with the proper kernel in order to adjust the PSF in the mosaic to match that of the specified JWST detector and filter. Note that this can only be done in cases where the mosaic PSF's FWHM is smaller than the JWST PSF's FWHM, otherwise we would be sharpening the image. If you attempt to run the code in a situation like that, an exception will be raised.

After convolution, the subarray is resampled onto the JWST pixel grid. Resample is essentially the same as Astrodrizzle's blot functionality.

The resampled image is then added to the previously created extended source catalog (or kept in its own catalog if template_cat is None). This leads to an extended source catalog that is specific to the input yaml file used to control the cropping and resampleing. This extended source catalog is added to the yaml file so that when the simulated data are created, it will be used.

In [ ]:
yam.yaml_files

In this case, with the mosaic image created from HST/ACS F814W data, with a FWHM of 0.09", we cannot run the code for the NIRCam shortwave detectors (B1 - B4) with F150W, because the corresponding FWHM for that is smaller than 0.09". However, we can run the code for the NIRCam longwave detector (B5) with the F444W filter, where the FWHM is larger than 0.09".

In [ ]:
for yfile in [yam.yaml_files[-1]]:
    
    # Read in the yaml file so that we know RA, Dec, PAV3
    # of the exposure
    with open(yfile) as file_obj:
        params = yaml.safe_load(file_obj)
        
    ra = params['Telescope']['ra']
    dec = params['Telescope']['dec']
    pav3 = params['Telescope']['rotation']

    # Define the output files and directories
    sim_data_dir = params['Output']['directory']
    simulated_filename = params['Output']['file']
    crop_file = simulated_filename.replace('.fits', '_cropped_from_mosaic.fits')
    crop_file = os.path.join(sim_data_dir, crop_file)
    blot_file = simulated_filename.replace('.fits', '_blotted_seed_image.fits')
    
    # Crop from the mosaic and resample for the desired detector/aperture
    seed = ImgSeed(paramfile=yfile, mosaic_file=mosaicfile, cropped_file=crop_file,
                   outdir=sim_data_dir, blotted_file=blot_file, mosaic_fwhm=mosaic_fwhm,
                   mosaic_fwhm_units='arcsec', gaussian_psf=False)
    seed.crop_and_blot()

    # Now add the resampled file to the extended source catalog template and
    # save as a separate catalog file
    
    # Need to add a magnitude entry for each filter/pupil
    mosaic_f150w_mag = ['None']
    mosaic_f444w_mag = ['None']
    
    # Create the catalog containing only the resampled image
    blotted_image_full_path = os.path.join(sim_data_dir, blot_file)
    extended_catalog_file = simulated_filename.replace('.fits', '_extended_sources.cat')
    ext_cat = ExtendedCatalog(filenames=[blotted_image_full_path], ra=[ra], dec=[dec], position_angle=[pav3])
    ext_cat.add_magnitude_column(mosaic_f150w_mag, instrument='nircam', filter_name=filter1)
    ext_cat.add_magnitude_column(mosaic_f444w_mag, instrument='nircam', filter_name=filter2)

    # Combine the resampled image catalog and the template catalog
    if template_cat is not None:
        combined_cat = combine_catalogs(ext_cat, template_cat)
        combined_cat.save(extended_catalog_file)
    else:
        ext_cat.save(extended_catalog_file)

    # Now add this extended source catalog to the yaml file
    params['simSignals']['extended'] = extended_catalog_file

    # Save the updated yaml file
    with open(yfile, 'w') as file_obj:
        dump = yaml.dump(params, default_flow_style=False)
        file_obj.write(dump)

---
<a id='create_data_catalog'></a>
### Create the simulated data

Run the imaging simulator using the yaml files. Again in this case we run only the case with the NIRCam longwave detector.

In [ ]:
for yfile in [yam.yaml_files[-1]]:
    sim = ImgSim(paramfile=yfile)
    sim.create()

---
<a id='examine_data_catalog'></a>
### Look at simulated data

In [ ]:
def show(array,title,min=0,max=1000):
    plt.figure(figsize=(12,12))
    plt.imshow(array,clim=(min,max), origin='lower')
    plt.title(title)
    plt.colorbar().set_label('DN$^{-}$/s')

#### Seed image

Note that the sources are more difficult to see than you might expect. This is because background signal has been added to the data. This will be in addition to any background signal present in the original mosaic image. As with any Mirage simulation, the level of background signal can be controlled using the `bkgdrate` parameter in the yaml file.

The three stamp images added on top of the mosaic are visible in a diagonal line in the center of the image, going from upper left to lower right.

In [ ]:
# Look at the noiseless seed image
show(sim.seedimage,'Seed Image', max=0.4)

Zoom in on the center, where the three added stamp images are. The sources are in the center, the upper left corner, and the lower right corner.

In [ ]:
show(sim.seedimage[700: 1300, 700: 1300],'Seed Image', max=0.6)

---
<a id='resample_seed'></a>
## Use resampled image as a seed image

In this case, rather than adding the cropped and resampled images to extended source catalogs which are then used in the simulated data generation, we instead use the cropped and resampled images as Mirage's seed images. This means that no sources other than the mosaic image can be used. After cropping and resampling, Mirage's dark current prep and observation generator are run, creating the final simulated data directly.

In [ ]:
mosaicfile = 'hlsp_candels_hst_acs_gs-tot-sect23_f814w_v1.0_drz.fits'
xml_file = 'extended_object_test.xml'
pointing_file = xml_file.replace('.xml', '.pointing')

---
<a id='yaml_generator_seed'></a>
### Run yaml_generator to create Mirage input yaml files

User-inputs to the yaml generator. Note that you cannot use a catalogs input here to add extra
point sources or galaxies.

In [ ]:
cr = {'library': 'SUNMAX', 'scale': 1.0}
dates = '2019-5-25'
background = 'low'
pav3 = 0.0

Run the yaml generator

In [ ]:
yam = yaml_generator.SimInput(xml_file, pointing_file, verbose=True,
                              output_dir='yamls',
                              cosmic_rays=cr,
                              background=background, roll_angle=pav3, dates=dates,
                              simdata_output_dir='simdata',
                              datatype='raw')
yam.use_linearized_darks = True
yam.create_inputs()

---
<a id='create_data_seed'></a>
### Create the simulted data

In this step, crop a roughly detector-sized subarray from the mosaic image at the location specified in the yaml file. Convolve the subarray with the proper kernel in order to adjust the PSF in the mosaic to match that of the specified JWST detector and filter. Note that this can only be done in cases where the mosaic PSF's FWHM is smaller than the JWST PSF's FWHM, otherwise we would be sharpening the image. If you attempt to run the code in a situation like that, an exception will be raised.

After convolution, the subarray is resampled onto the JWST pixel grid. Resample is essentially the same as Astrodrizzle's blot functionality.

The resampled image serves as Mirage's seed image. The dark_prep and observation generator steps are then run to complete the simulated data generation.

As in the previous method, we run only the NIRCam longwave detector data here, as this is the only case where the NIRCam PSF FWHM is larger than the mosaic PSF FWHM.

In [ ]:
for yfile in [yam.yaml_files[-1]]:
    
    # Read in the yaml file so that we know RA, Dec, PAV3
    with open(yfile) as file_obj:
        params = yaml.safe_load(file_obj)
        
    # Define output filenames and directories
    sim_data_dir = params['Output']['directory']
    simulated_filename = params['Output']['file']
    crop_file = simulated_filename.replace('.fits', '_cropped_from_mosaic.fits')
    crop_file = os.path.join(sim_data_dir, crop_file)
    blot_file = simulated_filename.replace('.fits', '_blotted_seed_image.fits')
        
    # Crop from the mosaic and then resample the image
    seed = ImgSeed(paramfile=yfile, mosaic_file=mosaicfile, cropped_file=crop_file,
                   outdir=sim_data_dir, blotted_file=blot_file, mosaic_fwhm=mosaic_fwhm,
                   mosaic_fwhm_units='arcsec', gaussian_psf=False)
    seed.crop_and_blot()
    
    # Run dark_prep
    dark = DarkPrep()
    dark.paramfile = yfile
    dark.prepare()

    # Run the observation generator
    obs = Observation()
    obs.paramfile = yfile    
    obs.seed = seed.seed_image
    obs.segmap = seed.seed_segmap
    obs.seedheader = seed.seedinfo
    obs.linDark = dark.prepDark
    obs.create()

---
<a id='examine_data_seed'></a>
### Look at simulated data

In [ ]:
seed_file = 'simdata/jw00042001001_01101_00001_nrcb5_uncal_blotted_seed_image.fits'

In [ ]:
seed_image = fits.getdata(seed_file)

In this case, no background has been added to the mosaic image because the cutout from the mosaic is being used directly as the seed image. The only background signal present is that present in the original image.

In [ ]:
# Look at the noiseless seed image
show(seed_image,'Seed Image',max=0.03)